# Module 03: Building Neural Networks with NumPy (From Scratch)

**Difficulty**: ⭐⭐⭐ (Advanced)
**Estimated Time**: 60-75 minutes
**Prerequisites**: 
- Module 00: Introduction to Neural Networks
- Module 01: Perceptrons and Activation Functions
- Module 02: Backpropagation and Gradient Descent
- Strong Python and NumPy skills
- Understanding of object-oriented programming

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Implement** a complete multi-layer perceptron (MLP) from scratch using only NumPy
2. **Design** modular neural network classes with forward and backward methods
3. **Train** neural networks on non-linearly separable datasets (XOR, circles)
4. **Debug** neural network training by monitoring loss and gradients
5. **Compare** your implementation with scikit-learn's MLPClassifier
6. **Understand** every component: initialization, forward pass, backward pass, optimization

## 1. Setup and Imports

In [ ]:
# Standard libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# Set random seeds for reproducibility
np.random.seed(42)

# Configure plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Setup complete!")
print(f"NumPy version: {np.__version__}")

## 2. Activation Functions Module

First, let's create a reusable module for activation functions and their derivatives.

In [ ]:
class Activation:
    """
    Collection of activation functions and their derivatives.
    Each activation is a class method that can be called without instantiation.
    """
    
    @staticmethod
    def sigmoid(z):
        """Sigmoid activation: 1 / (1 + e^(-z))"""
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    @staticmethod
    def sigmoid_derivative(z):
        """Derivative of sigmoid: sigmoid(z) * (1 - sigmoid(z))"""
        s = Activation.sigmoid(z)
        return s * (1 - s)
    
    @staticmethod
    def relu(z):
        """ReLU activation: max(0, z)"""
        return np.maximum(0, z)
    
    @staticmethod
    def relu_derivative(z):
        """Derivative of ReLU: 1 if z > 0, else 0"""
        return (z > 0).astype(float)
    
    @staticmethod
    def tanh(z):
        """Tanh activation"""
        return np.tanh(z)
    
    @staticmethod
    def tanh_derivative(z):
        """Derivative of tanh: 1 - tanh^2(z)"""
        return 1 - np.tanh(z) ** 2
    
    @staticmethod
    def softmax(z):
        """
        Softmax activation for multi-class classification.
        Numerically stable implementation.
        """
        exp_z = np.exp(z - np.max(z, axis=0, keepdims=True))
        return exp_z / np.sum(exp_z, axis=0, keepdims=True)
    
    @staticmethod
    def get_activation(name):
        """Get activation function by name."""
        activations = {
            'sigmoid': (Activation.sigmoid, Activation.sigmoid_derivative),
            'relu': (Activation.relu, Activation.relu_derivative),
            'tanh': (Activation.tanh, Activation.tanh_derivative)
        }
        if name not in activations:
            raise ValueError(f"Unknown activation: {name}. Choose from {list(activations.keys())}")
        return activations[name]

print("Activation functions module loaded.")
print("Available: sigmoid, relu, tanh, softmax")

## 3. Neural Network Layer Class

Let's build a fully-connected (dense) layer class that handles:
- Weight initialization
- Forward propagation
- Backward propagation

In [ ]:
class DenseLayer:
    """
    Fully-connected neural network layer.
    
    Parameters:
    -----------
    input_size : int
        Number of input features
    output_size : int
        Number of neurons in this layer
    activation : str
        Activation function name ('sigmoid', 'relu', 'tanh')
    """
    
    def __init__(self, input_size, output_size, activation='relu'):
        self.input_size = input_size
        self.output_size = output_size
        self.activation_name = activation
        
        # Get activation function and its derivative
        self.activation, self.activation_derivative = Activation.get_activation(activation)
        
        # Initialize weights using He initialization (good for ReLU)
        # Weights shape: (output_size, input_size)
        self.W = np.random.randn(output_size, input_size) * np.sqrt(2.0 / input_size)
        
        # Initialize biases to zero
        # Biases shape: (output_size, 1)
        self.b = np.zeros((output_size, 1))
        
        # Cache for backward pass
        self.cache = {}
    
    def forward(self, A_prev):
        """
        Forward propagation through the layer.
        
        Parameters:
        -----------
        A_prev : ndarray, shape (input_size, m)
            Activations from previous layer (or input)
            m = number of samples
        
        Returns:
        --------
        A : ndarray, shape (output_size, m)
            Activations of this layer
        """
        # Linear transformation: Z = W @ A_prev + b
        Z = np.dot(self.W, A_prev) + self.b
        
        # Apply activation function
        A = self.activation(Z)
        
        # Cache values needed for backward pass
        self.cache = {
            'A_prev': A_prev,
            'Z': Z,
            'A': A
        }
        
        return A
    
    def backward(self, dA):
        """
        Backward propagation through the layer.
        
        Parameters:
        -----------
        dA : ndarray, shape (output_size, m)
            Gradient of loss with respect to this layer's activation
        
        Returns:
        --------
        dA_prev : ndarray, shape (input_size, m)
            Gradient to pass to previous layer
        """
        # Retrieve cached values
        A_prev = self.cache['A_prev']
        Z = self.cache['Z']
        m = A_prev.shape[1]  # Number of samples
        
        # Compute dZ: gradient of loss w.r.t. pre-activation
        # dZ = dA * activation'(Z)
        dZ = dA * self.activation_derivative(Z)
        
        # Compute gradients for parameters
        # dW = (1/m) * dZ @ A_prev.T
        self.dW = (1 / m) * np.dot(dZ, A_prev.T)
        
        # db = (1/m) * sum of dZ across samples
        self.db = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
        
        # Compute gradient for previous layer
        # dA_prev = W.T @ dZ
        dA_prev = np.dot(self.W.T, dZ)
        
        return dA_prev
    
    def update_parameters(self, learning_rate):
        """
        Update parameters using gradient descent.
        
        Parameters:
        -----------
        learning_rate : float
            Learning rate for gradient descent
        """
        self.W -= learning_rate * self.dW
        self.b -= learning_rate * self.db

print("DenseLayer class defined.")
print("Features: forward propagation, backpropagation, parameter updates")

## 4. Complete Neural Network Class

Now let's build a complete neural network that can have multiple layers.

In [ ]:
class NeuralNetwork:
    """
    Multi-layer neural network for binary classification.
    
    Example usage:
    ```python
    nn = NeuralNetwork()
    nn.add_layer(2, 4, 'relu')      # Input: 2, Hidden: 4 neurons
    nn.add_layer(4, 4, 'relu')      # Hidden: 4 neurons
    nn.add_layer(4, 1, 'sigmoid')   # Output: 1 neuron
    nn.compile()
    nn.fit(X_train, y_train, epochs=1000, learning_rate=0.01)
    ```
    """
    
    def __init__(self):
        self.layers = []
        self.compiled = False
        self.history = {
            'loss': [],
            'accuracy': []
        }
    
    def add_layer(self, input_size, output_size, activation='relu'):
        """
        Add a dense layer to the network.
        
        Parameters:
        -----------
        input_size : int
            Number of input features to this layer
        output_size : int
            Number of neurons in this layer
        activation : str
            Activation function
        """
        layer = DenseLayer(input_size, output_size, activation)
        self.layers.append(layer)
    
    def compile(self):
        """Compile the network (validates architecture)."""
        if len(self.layers) == 0:
            raise ValueError("No layers added! Use add_layer() first.")
        
        self.compiled = True
        print("Network compiled successfully!")
        print(f"Architecture: {self.get_architecture()}")
    
    def get_architecture(self):
        """Get a string representation of network architecture."""
        arch = []
        for i, layer in enumerate(self.layers):
            arch.append(f"Layer {i+1}: {layer.input_size} → {layer.output_size} ({layer.activation_name})")
        return "\n  ".join(arch)
    
    def forward(self, X):
        """
        Forward propagation through entire network.
        
        Parameters:
        -----------
        X : ndarray, shape (n_features, m)
            Input data
        
        Returns:
        --------
        A : ndarray
            Final activations (predictions)
        """
        A = X
        for layer in self.layers:
            A = layer.forward(A)
        return A
    
    def compute_loss(self, Y_true, Y_pred):
        """
        Compute binary cross-entropy loss.
        
        Parameters:
        -----------
        Y_true : ndarray, shape (1, m)
            True labels
        Y_pred : ndarray, shape (1, m)
            Predicted probabilities
        
        Returns:
        --------
        loss : float
            Binary cross-entropy loss
        """
        m = Y_true.shape[1]
        epsilon = 1e-15  # Prevent log(0)
        
        Y_pred = np.clip(Y_pred, epsilon, 1 - epsilon)
        loss = -(1/m) * np.sum(Y_true * np.log(Y_pred) + 
                              (1 - Y_true) * np.log(1 - Y_pred))
        return loss
    
    def backward(self, Y_true, Y_pred):
        """
        Backward propagation through entire network.
        
        Parameters:
        -----------
        Y_true : ndarray, shape (1, m)
            True labels
        Y_pred : ndarray, shape (1, m)
            Predicted probabilities
        """
        # Gradient of loss w.r.t. final activation
        # For binary cross-entropy + sigmoid: dA = Y_pred - Y_true
        epsilon = 1e-15
        Y_pred = np.clip(Y_pred, epsilon, 1 - epsilon)
        dA = -(Y_true / Y_pred) + (1 - Y_true) / (1 - Y_pred)
        
        # Backward through all layers
        for layer in reversed(self.layers):
            dA = layer.backward(dA)
    
    def update_parameters(self, learning_rate):
        """Update all layer parameters."""
        for layer in self.layers:
            layer.update_parameters(learning_rate)
    
    def fit(self, X, Y, epochs=1000, learning_rate=0.01, verbose=True):
        """
        Train the neural network.
        
        Parameters:
        -----------
        X : ndarray, shape (n_features, m)
            Training data
        Y : ndarray, shape (1, m)
            Training labels
        epochs : int
            Number of training iterations
        learning_rate : float
            Learning rate for gradient descent
        verbose : bool
            Whether to print progress
        """
        if not self.compiled:
            raise ValueError("Network not compiled! Call compile() first.")
        
        self.history['loss'] = []
        self.history['accuracy'] = []
        
        for epoch in range(epochs):
            # Forward propagation
            Y_pred = self.forward(X)
            
            # Compute loss
            loss = self.compute_loss(Y, Y_pred)
            
            # Compute accuracy
            predictions = (Y_pred > 0.5).astype(int)
            accuracy = np.mean(predictions == Y)
            
            # Store history
            self.history['loss'].append(loss)
            self.history['accuracy'].append(accuracy)
            
            # Backward propagation
            self.backward(Y, Y_pred)
            
            # Update parameters
            self.update_parameters(learning_rate)
            
            # Print progress
            if verbose and (epoch % 100 == 0 or epoch == epochs - 1):
                print(f"Epoch {epoch:4d}/{epochs}: Loss = {loss:.4f}, "
                      f"Accuracy = {accuracy * 100:.2f}%")
    
    def predict(self, X):
        """
        Make predictions on new data.
        
        Parameters:
        -----------
        X : ndarray, shape (n_features, m)
            Input data
        
        Returns:
        --------
        predictions : ndarray, shape (1, m)
            Binary predictions (0 or 1)
        """
        Y_pred = self.forward(X)
        return (Y_pred > 0.5).astype(int)
    
    def predict_proba(self, X):
        """
        Get prediction probabilities.
        
        Parameters:
        -----------
        X : ndarray, shape (n_features, m)
            Input data
        
        Returns:
        --------
        probabilities : ndarray, shape (1, m)
            Predicted probabilities
        """
        return self.forward(X)

print("NeuralNetwork class defined.")
print("Features: add_layer, compile, fit, predict, predict_proba")

## 5. Test on XOR Problem

Let's test our network on the classic XOR problem - a problem that single-layer perceptrons cannot solve!

In [ ]:
print("=" * 70)
print("TESTING ON XOR PROBLEM")
print("=" * 70)

# Create XOR dataset
X_xor = np.array([[0, 0, 1, 1],
                  [0, 1, 0, 1]])  # Shape: (2, 4)
Y_xor = np.array([[0, 1, 1, 0]])  # Shape: (1, 4)

print("\nXOR Dataset:")
print("Inputs:")
print(X_xor.T)
print("\nOutputs:")
print(Y_xor.T)

# Build network
print("\n" + "=" * 70)
print("Building Neural Network...")
print("=" * 70)

nn_xor = NeuralNetwork()
nn_xor.add_layer(2, 4, 'relu')      # Input: 2, Hidden: 4
nn_xor.add_layer(4, 1, 'sigmoid')   # Hidden: 4, Output: 1
nn_xor.compile()

# Train
print("\nTraining...")
nn_xor.fit(X_xor, Y_xor, epochs=2000, learning_rate=0.1, verbose=True)

# Test
print("\n" + "=" * 70)
print("FINAL RESULTS")
print("=" * 70)

predictions = nn_xor.predict(X_xor)
probabilities = nn_xor.predict_proba(X_xor)

print("\nInput | True | Predicted | Probability | Correct")
print("-" * 60)
for i in range(4):
    x1, x2 = X_xor[:, i]
    y_true = Y_xor[0, i]
    y_pred = predictions[0, i]
    prob = probabilities[0, i]
    correct = "✓" if y_pred == y_true else "✗"
    print(f"[{int(x1)}, {int(x2)}]  |  {int(y_true)}   |     {int(y_pred)}     |   {prob:.4f}    | {correct}")

accuracy = np.mean(predictions == Y_xor)
print("-" * 60)
print(f"Final Accuracy: {accuracy * 100:.2f}%")
print("=" * 70)

if accuracy == 1.0:
    print("\n🎉 SUCCESS! XOR problem solved perfectly!")
    print("This proves that multi-layer networks can solve non-linear problems!")
else:
    print("\n⚠️ Network didn't fully converge. Try:")
    print("  - Training for more epochs")
    print("  - Adjusting learning rate")
    print("  - Re-running (different initialization)")

In [ ]:
# Plot training history for XOR

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot loss
ax1.plot(nn_xor.history['loss'], linewidth=2, color='blue')
ax1.set_xlabel('Epoch', fontsize=12, weight='bold')
ax1.set_ylabel('Loss (Binary Cross-Entropy)', fontsize=12, weight='bold')
ax1.set_title('Training Loss', fontsize=13, weight='bold')
ax1.grid(True, alpha=0.3)

# Plot accuracy
ax2.plot(nn_xor.history['accuracy'], linewidth=2, color='green')
ax2.axhline(y=1.0, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Perfect Accuracy')
ax2.set_xlabel('Epoch', fontsize=12, weight='bold')
ax2.set_ylabel('Accuracy', fontsize=12, weight='bold')
ax2.set_title('Training Accuracy', fontsize=13, weight='bold')
ax2.set_ylim(0, 1.1)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.suptitle('XOR Problem - Training Progress', fontsize=15, weight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 6. Test on Circles Dataset

Now let's tackle a more challenging non-linear problem: concentric circles.

In [ ]:
# Generate circles dataset
X_circles, y_circles = make_circles(n_samples=1000, noise=0.1, factor=0.5, random_state=42)

# Reshape for our network (features as rows)
X_circles = X_circles.T  # Shape: (2, 1000)
y_circles = y_circles.reshape(1, -1)  # Shape: (1, 1000)

print("Circles Dataset:")
print(f"  Shape: {X_circles.shape}")
print(f"  Classes: {np.unique(y_circles)}")
print(f"  Class distribution: {np.bincount(y_circles[0])}")

# Visualize dataset
plt.figure(figsize=(8, 8))
plt.scatter(X_circles[0, y_circles[0] == 0], X_circles[1, y_circles[0] == 0],
           c='blue', s=30, alpha=0.6, edgecolors='k', linewidth=0.5, label='Class 0')
plt.scatter(X_circles[0, y_circles[0] == 1], X_circles[1, y_circles[0] == 1],
           c='red', s=30, alpha=0.6, edgecolors='k', linewidth=0.5, label='Class 1')
plt.xlabel('Feature 1', fontsize=12, weight='bold')
plt.ylabel('Feature 2', fontsize=12, weight='bold')
plt.title('Circles Dataset (Non-linearly Separable)', fontsize=14, weight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
# Build and train network on circles

print("=" * 70)
print("BUILDING NETWORK FOR CIRCLES DATASET")
print("=" * 70)

nn_circles = NeuralNetwork()
nn_circles.add_layer(2, 16, 'relu')     # Input: 2, Hidden: 16
nn_circles.add_layer(16, 8, 'relu')     # Hidden: 8
nn_circles.add_layer(8, 1, 'sigmoid')   # Output: 1
nn_circles.compile()

print("\nTraining...")
nn_circles.fit(X_circles, y_circles, epochs=1000, learning_rate=0.1, verbose=True)

# Evaluate
predictions = nn_circles.predict(X_circles)
accuracy = np.mean(predictions == y_circles)

print("\n" + "=" * 70)
print(f"FINAL ACCURACY: {accuracy * 100:.2f}%")
print("=" * 70)

In [ ]:
def plot_decision_boundary(model, X, y, title="Decision Boundary"):
    """
    Plot decision boundary for a binary classifier.
    
    Parameters:
    -----------
    model : NeuralNetwork or sklearn model
        Trained model with predict method
    X : ndarray, shape (2, m)
        Input data (2 features)
    y : ndarray, shape (1, m) or (m,)
        True labels
    title : str
        Plot title
    """
    # Create mesh grid
    h = 0.02  # Step size
    x_min, x_max = X[0].min() - 0.5, X[0].max() + 0.5
    y_min, y_max = X[1].min() - 0.5, X[1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                        np.arange(y_min, y_max, h))
    
    # Predict for each point in mesh
    mesh_input = np.c_[xx.ravel(), yy.ravel()].T  # Shape: (2, n_points)
    
    if isinstance(model, NeuralNetwork):
        Z = model.predict(mesh_input)
        Z = Z.reshape(xx.shape)
    else:  # sklearn model
        Z = model.predict(mesh_input.T)
        Z = Z.reshape(xx.shape)
    
    # Plot decision boundary
    plt.figure(figsize=(10, 8))
    plt.contourf(xx, yy, Z, alpha=0.3, levels=1, colors=['blue', 'red'])
    plt.contour(xx, yy, Z, levels=[0.5], colors='black', linewidths=2)
    
    # Flatten y if needed
    y_flat = y.ravel() if len(y.shape) > 1 else y
    
    # Plot data points
    plt.scatter(X[0, y_flat == 0], X[1, y_flat == 0],
               c='blue', s=30, alpha=0.6, edgecolors='k', 
               linewidth=0.5, label='Class 0')
    plt.scatter(X[0, y_flat == 1], X[1, y_flat == 1],
               c='red', s=30, alpha=0.6, edgecolors='k', 
               linewidth=0.5, label='Class 1')
    
    plt.xlabel('Feature 1', fontsize=12, weight='bold')
    plt.ylabel('Feature 2', fontsize=12, weight='bold')
    plt.title(title, fontsize=14, weight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Plot decision boundary
plot_decision_boundary(nn_circles, X_circles, y_circles,
                      title=f"Our Neural Network Decision Boundary (Acc: {accuracy*100:.1f}%)")

## 7. Comparison with Scikit-Learn

Let's compare our implementation with scikit-learn's MLPClassifier.

In [ ]:
print("=" * 70)
print("COMPARISON WITH SCIKIT-LEARN MLPClassifier")
print("=" * 70)

# Prepare data for sklearn (expects features as columns)
X_sklearn = X_circles.T
y_sklearn = y_circles.ravel()

# Build sklearn model with similar architecture
mlp_sklearn = MLPClassifier(
    hidden_layer_sizes=(16, 8),  # Same as our network
    activation='relu',
    solver='sgd',               # Stochastic gradient descent
    learning_rate_init=0.1,     # Same learning rate
    max_iter=1000,              # Same epochs
    random_state=42,
    batch_size=X_sklearn.shape[0]  # Full batch (like ours)
)

print("\nTraining sklearn MLPClassifier...")
mlp_sklearn.fit(X_sklearn, y_sklearn)

# Evaluate
sklearn_accuracy = mlp_sklearn.score(X_sklearn, y_sklearn)

print("\n" + "=" * 70)
print("RESULTS COMPARISON")
print("=" * 70)
print(f"Our Implementation:    {accuracy * 100:.2f}%")
print(f"Sklearn MLPClassifier: {sklearn_accuracy * 100:.2f}%")
print(f"Difference:            {abs(accuracy - sklearn_accuracy) * 100:.2f}%")
print("=" * 70)

if abs(accuracy - sklearn_accuracy) < 0.05:
    print("\n✓ Great! Our implementation performs similarly to sklearn!")
else:
    print("\n⚠️ Note: Some difference is expected due to:")
    print("  - Different weight initializations")
    print("  - Numerical precision differences")
    print("  - Optimization details")

In [ ]:
# Plot sklearn's decision boundary for comparison
plot_decision_boundary(mlp_sklearn, X_circles, y_circles,
                      title=f"Sklearn MLPClassifier Decision Boundary (Acc: {sklearn_accuracy*100:.1f}%)")

## 8. Exercises

### Exercise 1: Implement Different Layer Sizes

Experiment with different network architectures on the circles dataset:

1. **Shallow and wide**: 2 → 32 → 1
2. **Deep and narrow**: 2 → 8 → 8 → 8 → 1
3. **Pyramid**: 2 → 16 → 8 → 4 → 1

**Questions**:
- Which architecture achieves the highest accuracy?
- Which trains fastest?
- Which generalizes best (test on a validation set)?

In [ ]:
# Exercise 1: Your solution here

# TODO: Split data into train/validation sets

# TODO: Test architecture 1: Shallow and wide
# nn1 = NeuralNetwork()
# ...

# TODO: Test architecture 2: Deep and narrow
# nn2 = NeuralNetwork()
# ...

# TODO: Test architecture 3: Pyramid
# nn3 = NeuralNetwork()
# ...

# TODO: Compare results in a table

In [ ]:
# Solution to Exercise 1

from sklearn.model_selection import train_test_split

# Split data
X_train, X_val, y_train, y_val = train_test_split(
    X_circles.T, y_circles.T, test_size=0.2, random_state=42
)
X_train, X_val = X_train.T, X_val.T
y_train, y_val = y_train.T, y_val.T

print(f"Training set: {X_train.shape[1]} samples")
print(f"Validation set: {X_val.shape[1]} samples")

architectures = [
    ('Shallow & Wide', [(2, 32, 'relu'), (32, 1, 'sigmoid')]),
    ('Deep & Narrow', [(2, 8, 'relu'), (8, 8, 'relu'), (8, 8, 'relu'), (8, 1, 'sigmoid')]),
    ('Pyramid', [(2, 16, 'relu'), (16, 8, 'relu'), (8, 4, 'relu'), (4, 1, 'sigmoid')])
]

results = []

for name, layers in architectures:
    print(f"\n{'='*70}")
    print(f"Testing: {name}")
    print(f"{'='*70}")
    
    # Build network
    nn = NeuralNetwork()
    for layer_config in layers:
        nn.add_layer(*layer_config)
    nn.compile()
    
    # Train
    import time
    start_time = time.time()
    nn.fit(X_train, y_train, epochs=1000, learning_rate=0.1, verbose=False)
    training_time = time.time() - start_time
    
    # Evaluate
    train_acc = np.mean(nn.predict(X_train) == y_train)
    val_acc = np.mean(nn.predict(X_val) == y_val)
    
    results.append({
        'name': name,
        'train_acc': train_acc,
        'val_acc': val_acc,
        'time': training_time
    })
    
    print(f"Training Accuracy: {train_acc*100:.2f}%")
    print(f"Validation Accuracy: {val_acc*100:.2f}%")
    print(f"Training Time: {training_time:.2f}s")

# Summary table
print("\n" + "="*80)
print("ARCHITECTURE COMPARISON")
print("="*80)
print(f"{'Architecture':<20} | {'Train Acc':<10} | {'Val Acc':<10} | {'Time (s)':<10}")
print("-"*80)
for r in results:
    print(f"{r['name']:<20} | {r['train_acc']*100:>9.2f}% | {r['val_acc']*100:>9.2f}% | {r['time']:>9.2f}s")
print("="*80)

# Find best
best_val = max(results, key=lambda x: x['val_acc'])
fastest = min(results, key=lambda x: x['time'])

print(f"\nBest Validation Accuracy: {best_val['name']} ({best_val['val_acc']*100:.2f}%)")
print(f"Fastest Training: {fastest['name']} ({fastest['time']:.2f}s)")

### Exercise 2: Implement Moons Dataset

Train your neural network on the "moons" dataset (another non-linear problem).

**Tasks**:
1. Generate moons dataset using `make_moons(n_samples=1000, noise=0.2)`
2. Design an appropriate architecture
3. Train the network
4. Visualize the decision boundary
5. Achieve >95% accuracy

In [ ]:
# Exercise 2: Your solution here

# TODO: Generate moons dataset
# X_moons, y_moons = make_moons(...)

# TODO: Visualize the dataset

# TODO: Build and train network

# TODO: Evaluate and plot decision boundary

In [ ]:
# Solution to Exercise 2

# Generate dataset
X_moons, y_moons = make_moons(n_samples=1000, noise=0.2, random_state=42)
X_moons = X_moons.T
y_moons = y_moons.reshape(1, -1)

print("Moons Dataset Generated")
print(f"Shape: {X_moons.shape}")

# Visualize
plt.figure(figsize=(8, 8))
plt.scatter(X_moons[0, y_moons[0] == 0], X_moons[1, y_moons[0] == 0],
           c='blue', s=30, alpha=0.6, edgecolors='k', linewidth=0.5, label='Class 0')
plt.scatter(X_moons[0, y_moons[0] == 1], X_moons[1, y_moons[0] == 1],
           c='red', s=30, alpha=0.6, edgecolors='k', linewidth=0.5, label='Class 1')
plt.xlabel('Feature 1', fontsize=12, weight='bold')
plt.ylabel('Feature 2', fontsize=12, weight='bold')
plt.title('Moons Dataset', fontsize=14, weight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Build network
print("\nBuilding Neural Network...")
nn_moons = NeuralNetwork()
nn_moons.add_layer(2, 8, 'relu')
nn_moons.add_layer(8, 8, 'relu')
nn_moons.add_layer(8, 1, 'sigmoid')
nn_moons.compile()

# Train
print("\nTraining...")
nn_moons.fit(X_moons, y_moons, epochs=1000, learning_rate=0.1, verbose=True)

# Evaluate
predictions = nn_moons.predict(X_moons)
accuracy = np.mean(predictions == y_moons)

print(f"\nFinal Accuracy: {accuracy * 100:.2f}%")

if accuracy >= 0.95:
    print("✓ Target achieved: >95% accuracy!")

# Plot decision boundary
plot_decision_boundary(nn_moons, X_moons, y_moons,
                      title=f"Moons Dataset - Decision Boundary (Acc: {accuracy*100:.1f}%)")

### Exercise 3: Add Momentum to Optimizer

Enhance the neural network by implementing momentum in the parameter updates.

**Momentum Update Rule**:
$$v_t = \beta v_{t-1} + (1-\beta) \nabla J$$
$$\theta_t = \theta_{t-1} - \alpha v_t$$

Where:
- $v_t$ = velocity (exponentially weighted average of gradients)
- $\beta$ = momentum coefficient (typically 0.9)
- $\alpha$ = learning rate

**Tasks**:
1. Modify `DenseLayer.update_parameters()` to include momentum
2. Test on circles dataset
3. Compare convergence speed with and without momentum

In [ ]:
# Exercise 3: Your solution here

# TODO: Create DenseLayerWithMomentum class
# Hint: Add v_W and v_b attributes for velocity

# TODO: Test with and without momentum on circles dataset

# TODO: Plot loss curves to compare convergence

## 9. Summary

Congratulations! You've built a complete neural network from scratch using only NumPy. This is a significant achievement that gives you deep understanding of how neural networks actually work.

### Key Accomplishments

1. **Implemented Core Components**
   - Activation functions (sigmoid, ReLU, tanh) with derivatives
   - Dense layer with forward and backward propagation
   - Complete neural network with multiple layers
   - Training loop with loss computation and optimization

2. **Solved Non-Linear Problems**
   - XOR problem (impossible for single perceptron)
   - Circles dataset (concentric patterns)
   - Achieved high accuracy on complex decision boundaries

3. **Validated Implementation**
   - Compared with scikit-learn's MLPClassifier
   - Results match professional implementations
   - Understanding of every detail

### Technical Insights

- **Weight Initialization**: He initialization prevents vanishing/exploding gradients
- **Batch Processing**: Matrix operations handle multiple samples efficiently
- **Gradient Flow**: Chain rule propagates gradients backward through layers
- **Loss Monitoring**: Tracking loss helps debug and tune training
- **Architecture Design**: Depth and width affect learning capacity

### What We Learned

**Forward Propagation**:
```
For each layer:
  Z = W @ A_prev + b
  A = activation(Z)
```

**Backward Propagation**:
```
For each layer (reversed):
  dZ = dA * activation'(Z)
  dW = (1/m) * dZ @ A_prev.T
  db = (1/m) * sum(dZ)
  dA_prev = W.T @ dZ
```

**Parameter Update**:
```
W = W - learning_rate * dW
b = b - learning_rate * db
```

### What's Next?

In **Module 04: Introduction to TensorFlow/Keras**, we'll:
- Learn professional deep learning frameworks
- Build networks much faster with high-level APIs
- Access GPU acceleration for large-scale training
- Train on real datasets like MNIST and Fashion-MNIST
- Use advanced features: callbacks, model saving, visualization

### Why This Matters

Building neural networks from scratch gives you:
- **Deep Understanding**: Know exactly what happens under the hood
- **Debugging Skills**: Can diagnose and fix training issues
- **Architecture Intuition**: Understand design trade-offs
- **Research Capability**: Can implement novel architectures
- **Interview Preparation**: Demonstrate mastery in technical interviews

### Additional Resources

**Code Examples**:
- Andrej Karpathy: "micrograd" (minimal autograd engine)
- Joel Grus: "neural network from scratch" (livecoding)

**Theory**:
- CS231n: "Neural Networks Part 1-3" (Stanford)
- "Neural Networks and Deep Learning" by Michael Nielsen

**Practice**:
- Implement more activation functions (ELU, SELU, Mish)
- Add regularization (L1, L2, dropout)
- Implement different optimizers (Adam, RMSprop)
- Build convolutional layers for images

---

**Ready to use professional tools?** Continue to **Module 04: Introduction to TensorFlow/Keras**!